### Where is Kari Lake getting her contributions?


In [ ]:
import pandas as pd
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import json
from google.colab import files

per_page = '1000'
candidate_id = 'S4AZ00220'
cycle = '2024'
api_key = 'XXXXX'

url = f"https://api.open.fec.gov/v1/schedules/schedule_a/by_state/by_candidate/?page=1&per_page={per_page}&candidate_id={candidate_id}&cycle={cycle}&api_key={api_key}"

response = requests.get(url)
data = response.json()

data = data['results']
df = pd.DataFrame(data)



In [ ]:
# Do some cleanup

df = df[['count', 'state_full', 'total']]
df['total'] = pd.to_numeric(df['total'])
df['count'] = pd.to_numeric(df['count'])

df_totals = df

df_totals = df_totals.sort_values(by='total', ascending=False)
print(df_totals)

In [ ]:

url = "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json"

response = requests.get(url)
response.raise_for_status()  # Raise an exception if needed

gdf = gpd.read_file(response.text)

merged_gdf = gdf.merge(df_totals, left_on='name', right_on='state_full', how='left')


In [ ]:
# Create a choropleth map

fig, ax = plt.subplots(1, 1, figsize=(10, 6))  # Adjust figure size as needed


merged_gdf.plot(column='total',        # Column to visualize
                cmap='Greens',         # Choose a colormap
                linewidth=0.5,         # Border line width
                edgecolor='black',     # Border line color
                legend=True,           # Show legend
                ax=ax)

ax.set_title("Kari Lake's donors by state")  # Set title
ax.set_axis_off()

plt.show()

In [ ]:
# Integrate state populations to make a rate

state_pops = [{'state_full':'Pennsylvania','population':10353548},{'state_full':'California','population':30827105},{'state_full':'West Virginia','population':1432932},{'state_full':'Utah','population':2324051},{'state_full':'New York','population':16088135},{'state_full':'District of Columbia','population':575161},{'state_full':'Alaska','population':554003},{'state_full':'Florida','population':17339232},{'state_full':'South Carolina','population':4014460},{'state_full':'North Dakota','population':596093},{'state_full':'Maine','population':1110085},{'state_full':'Georgia','population':8220274},{'state_full':'Alabama','population':3917166},{'state_full':'New Hampshire','population':1120680},{'state_full':'Oregon','population':3370652},{'state_full':'Wyoming','population':441337},{'state_full':'Arizona','population':5541976},{'state_full':'Louisiana','population':3570548},{'state_full':'Indiana','population':5192579},{'state_full':'Idaho','population':1376400},{'state_full':'Connecticut','population':2869227},{'state_full':'Hawaii','population':1155905},{'state_full':'Illinois','population':9999469},{'state_full':'Massachusetts','population':5663723},{'state_full':'Texas','population':21866700},{'state_full':'Montana','population':850123},{'state_full':'Nebraska','population':1476127},{'state_full':'Ohio','population':9207562},{'state_full':'Colorado','population':4509576},{'state_full':'New Jersey','population':7281310},{'state_full':'Maryland','population':4815202},{'state_full':'Virginia','population':6745054},{'state_full':'Vermont','population':524482},{'state_full':'North Carolina','population':8155099},{'state_full':'Arkansas','population':2312273},{'state_full':'Washington','population':6024689},{'state_full':'Kansas','population':2229316},{'state_full':'Oklahoma','population':3010698},{'state_full':'Wisconsin','population':4612300},{'state_full':'Mississippi','population':2277599},{'state_full':'Missouri','population':4775612},{'state_full':'Michigan','population':7914602},{'state_full':'Rhode Island','population':887594},{'state_full':'Minnesota','population':4389033},{'state_full':'Iowa','population':2450103},{'state_full':'New Mexico','population':1638989},{'state_full':'Nevada','population':2413326},{'state_full':'Delaware','population':783543},{'state_full':'Puerto Rico','population':2724903},{'state_full':'Kentucky','population':3483900},{'state_full':'South Dakota','population':669255},{'state_full':'Tennessee','population':5384473}]

df_pops = pd.DataFrame(state_pops)

print(df_pops)


In [ ]:
# Merge DataFrames on 'state_name'

merged_df = pd.merge(df_totals, df_pops, on='state_full')

merged_df['rate_per_100k'] = (merged_df['total'] / merged_df['population']) * 100000

print(merged_df)


In [ ]:

# Remove Arizona and make a map again

mask = merged_df['state_full'] == 'Arizona'
merged_df = merged_df[~mask]

merged_gdf_rates = gdf.merge(merged_df, left_on='name', right_on='state_full', how='left')

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

merged_gdf_rates.plot(column='rate_per_100k',  # Column to visualize
                cmap='Greens',          # Choose a colormap
                linewidth=0.5,          # Border line width
                edgecolor='black',      # Border line color
                legend=True,            # Show legend
                ax=ax)

ax.set_title("Kari Lake's donors per 100k")
ax.set_axis_off()

plt.show()

In [ ]:

# Save the GeoDataFrame to a KML file and download
merged_gdf_rates.to_file("kari_lake_donations.geojson", driver='GeoJSON')

files.download("kari_lake_donations.geojson")


### Where is Ruben Gallego spending his money?

In [ ]:
import pandas as pd
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import json
from google.colab import files


base_url = "https://api.open.fec.gov/v1/schedules/schedule_b/by_recipient/"
params = {
    "per_page": 100,
    "cycle": 2024,
    "committee_id": "C00558627",
    "api_key": "XXXXXX"
}

all_results = []
page = 1

while True:
    params['page'] = page

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'results' not in data or not data['results']:
        break

    all_results.extend(data['results'])
    page += 1


df = pd.DataFrame(all_results)

In [ ]:
# What do we have? First, let's make a copy

df2 = df
print(df2)


In [ ]:
# Sort from largest to smallest

df2 = df2.sort_values(by='recipient_disbursement_percent', ascending=False)
df2

In [ ]:
df2 = df2[['recipient_name', 'recipient_disbursement_percent', 'total']].head()
df2